In [1]:
import pandas as pd
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split

# Constants
EPOCHS = 10
BATCH_SIZE = 256
CLASSES = 10
CHANNELS = 1
IMAGE_WIDTH, IMAGE_HEIGHT = 28, 28
VALIDATION_RATIO = 0.1

# Load data 
directory = '../input/digit-recognizer/'

train_all = pd.read_csv(directory + 'train.csv')

Y_train_all = train_all['label'] 
X_train_all = train_all.drop(labels = ["label"],axis = 1)

# Normalize data
X_train_all = X_train_all / 255.0

# Reshape data
X_train_all = X_train_all.values.reshape(-1, IMAGE_WIDTH, IMAGE_HEIGHT, CHANNELS)

# One hot label encoding
Y_train_all = to_categorical(Y_train_all, CLASSES)

# Splitting data 
X_train, X_val, Y_train, Y_val = train_test_split(X_train_all,
                                                      Y_train_all,
                                                      test_size=VALIDATION_RATIO,
                                                      shuffle = True)

In [2]:
from tensorflow import keras
from tensorflow.keras import layers

# Setup neural network model
model = keras.Sequential([
    layers.Conv2D(32, (5, 5), input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, CHANNELS), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Dropout(0.2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(CLASSES, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train model
history = model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=1)

In [9]:
import matplotlib.pyplot as plt
# %matplotlib inline

# Drawing plots
epochs = [i for i in range(1, len(history.history['loss'])+1)]

plt.plot(epochs, history.history['loss'], color='blue', label="Training loss")
plt.plot(epochs, history.history['val_loss'], color='red', label="Validation loss")
plt.legend(loc='best')
plt.title('training')
plt.xlabel('epoch')
plt.show()

plt.plot(epochs, history.history['accuracy'], color='blue', label="Training accuracy")
plt.plot(epochs, history.history['val_accuracy'], color='red', label="Validation accuracy")
plt.legend(loc='best')
plt.title('validation')
plt.xlabel('epoch')
plt.show()

In [13]:
import numpy as np

# Load test data
X_test = pd.read_csv(directory + 'test.csv')

# Normalize test data
X_test = X_test / 255.0

# Reshape data
X_test = X_test.values.reshape(-1, IMAGE_WIDTH, IMAGE_HEIGHT, CHANNELS)

results = model.predict(X_test)
results = np.argmax(results, axis = 1)
results = pd.Series(results, name="Label")

plt.imshow(X_test[4])

results

In [14]:
submission = pd.concat([pd.Series(range(1, len(X_test)+1),name = "ImageId"),results], axis = 1)

submission.to_csv("taras_yurts_mnist_submition.csv", index=False)